In [40]:
import random
import numpy as np

my_data_file = 'temp_data.npy'
data = np.load(my_data_file)

initial_pop_size = 100
mutation_rate = 0.05
num_generations = 5
chromosome_length = 2
num_survivors = 50

#---------------------------------------------------------
# GENERATION ZERO
#---------------------------------------------------------
# A() generates the first, random generation to populate
# our data structure of potential solution chromosomes.

# np.linspace generates 5000 evenly spaced numbers between
# -1 and 80. np.random.choice randomly picks pairs of 100
# numbers (initial population size)  from those and
# populates a 2-dimensional array with them. These
# represent the m and b value of each potential soultion.

# This function needs no input and the output is a random
# generation that can be assigned to a variable.
#---------------------------------------------------------

def A():
    return np.random.choice(np.linspace(-1,80,num = 5000),size=(initial_pop_size, chromosome_length),replace=False)
    
#---------------------------------------------------------
# COST/ERROR FUNCTION
#---------------------------------------------------------
# B() is the cost function, which in this particular case
# calculates the error (sum of squared deviations of pre- 
# dictions).

# This function sums up the deviations [y-(m*x+b)] and
# squares them, after which they are divided by k (the
# total number of data points) and that number is returned.

# The inputs of the function are the coefficients m and
# b and it outputs the error of said line.
#--------------------------------------------------------- 

def B(coefficients):
    k = data.shape[0]
    return (1./k) * np.sum([(data[j,1]-(coefficients[0] * data[j,0] + coefficients[1]))**2 for j in range(k)])

#---------------------------------------------------------
# FITNESS VECTOR
#---------------------------------------------------------
# C() is the fitness vector function. It returns an array
# with the x-value of a chromosome and the corresponding
# cost/error.

# With a for loop it numbers the entire current population's
# cost scores from 1 to 100 and outputs it as an array 
# of the dimensions [num,score].

# C() takes no input and returns the "fitness vector", a
# numbered array of the error scores.
#---------------------------------------------------------
    
def C():
    return np.array([np.array([x,B(current_pop[x])]) for x in range(current_pop.shape[0])]) 

#---------------------------------------------------------
# SELECTION OF SURVIVORS
#---------------------------------------------------------
# Function D() determines the survivors based on their
# cost scores (Roulette wheel selection).

# random_selection is a variable that contains 25 randomly
# picked numbers out of 100 (length of fitness_vector). 
# Best gets assigned the index of the lowest cost score
# out of this random selection, which is found using the
# np.argmin function. It then uses the fitness vector to 
# find the values of this "best" solution and adds them
# to the current_population array.

# D() has no inputs and outputs the array of the current
# population with one additional good (out of its pool)
# solution added each time it is called.
#---------------------------------------------------------
# IMPROVEMENTS
#---------------------------------------------------------
# Just like any other function in this assignment, D()
# should not call on global variables, but rather take
# them as input. If one of the variable names such as 
# num_survivors and fitness_vector were to be changed it
# would render the function useless. Eliminating the extra
# element of a fitness vector and adding the scores to 
# the original population as an additional column would
# also make this function's last part of indexes much 
# easier and more legible. For the sake of legibility this
# could also be split up into two by adding a placeholder
# variable inbetween. Additionally this function selects 
# its survivors from a subset of 1/4th the original
# population, picked at random, to ensure that it does
# not get stuck in a local minima. This could be improved,
# however, by for example picking half the new generation
# from the absolute fitest and the other half at random
# to ensure that good solutions do not get lost by chance.
#---------------------------------------------------------

def D():
    random_selection = np.random.choice(range(len(fitness_vector)), num_survivors/2, replace=False)
    best = np.argmin(fitness_vector[random_selection,1])
    return current_pop[int(fitness_vector[random_selection[best]][0])]

#---------------------------------------------------------
# CROSSOVER
#---------------------------------------------------------
# E() executes crossover of the population.

# duplicate_survivors is an array of zeros of  the length
# of the new population minus the amount of survivors. The
# for loop filles the x-column with four copies of each
# survivor and then "permutates" this population (randomly
# rearanges their order). It then does the same with the
# y-column and returns the resulting array as a matrix.

# E() again takes no particular input and it outputs our 
# new, cross-overed solution population.
#---------------------------------------------------------

def E():
    duplicate_survivors = np.zeros((((new_population.shape[0] - len(survivors))),survivors.shape[1]))
    for x in range(survivors.shape[1]):
        duplicate_survivors[:, x] = np.repeat(survivors[:, x], 4, axis=0)
        duplicate_survivors[:, x] = np.random.permutation(duplicate_survivors[:, x])
    return np.matrix(duplicate_survivors)

#---------------------------------------------------------
# MUTATION
#---------------------------------------------------------
# Function F() serves for random mutations in our solution
# population.

# The mutation function would iterated through the
# generation, generate a random integer between 1 and 0
# and if that integer is lower than the mutation rate
# defined at the top of this script it will replace it
# with a randomly generated number that was created in the
# same sample space as the original population. x_or_y 
# determines whether the m or b value of the chromosome
# will be changed. Mutations are important because by 
# randomly generating chromosomes we are limited to those
# solutions and through iteration might get stuck in a
# local minima. Mutation is able to push the possible
# solutions just enough by adding random ones occasionally
# that it allows tthem to get out of a local minima and
# eventually settle into the global minima.

# This function should take as input the current
# population and the mutation rate and output the mutated
# population with (in this particular case) 5% on mutated
# chromosomes.
#---------------------------------------------------------

def F(population, mutat_rate):
    for n in population:
        if mutat_rate >= random.random():
            x_or_y = random.randint(0,1)
            n[x_or_y] = np.random.choice(np.linspace(-1,80, num = 1000), size = 1, replace = False)
    return population    

#---------------------------------------------------------

#---------------------------------------------------------
# MAIN ALGORITHM
#---------------------------------------------------------
# Combining all the above defined functions this algorithm
# generates a population of random possible soultions for
# our regression line (m and b values). It evaluates all
# of these using the error functions and then finds the
# best fits to populate a new generation of survivors,
# shuffled from the survivors. This wway it narrows down
# on a better and better solution every generation. In
# the end it settles on the best solution after x numbers
# of generations and returns this value and its error
# score to us.
#---------------------------------------------------------

current_pop = A()
new_population = np.zeros((num_survivors * 5, current_pop.shape[1]))

for i in range(num_generations):
    
    fitness_vector = C()
    survivors = np.zeros((num_survivors, chromosome_length))
    for n in range(len(survivors)):
        survivors[n] = D()
    new_population[:len(survivors)] = survivors
    new_population[len(survivors):] = E()
    new_population = F(new_population, mutation_rate)
    
    current_pop = new_population
    new_population = np.zeros((num_survivors * 5, current_pop.shape[1]))

fitness_vector = C()
best_solution = current_pop[np.argmin(fitness_vector[:,1])]
print "The best solution is", best_solution
print "with error equal to approximately", B(best_solution)

#---------------------------------------------------------

#---------------------------------------------------------
# PARAMETER VARIATION
#---------------------------------------------------------
# While I oringinally thought that increasing the number 
# of generations would lead to an improvement in precision
# it turned out that added generations cannot correct for
# the solutions getting stuck in local minima without a 
# mutation function and thus the solutions would vary even
# more with increasing numbers of generations. 

# Parameters that increased this algorithm's precision
# drastically, however, are size of the inital population,
# as there are more solutions to pick from at the 
# beginning and with this I also slightly increased the
# number of survivors each generation, to accommodate for
# those extra solutions. Setting them to 400 and 100 re-
# spectively produced highly accurate results even as the
# number of generations stayed low.
#---------------------------------------------------------

The best solution is [  0.3934787   72.05405405]
with error equal to approximately 28.618943074


In [41]:
#---------------------------------------------------------
# IMPROVEMENTS TO THE CODE
#---------------------------------------------------------
# There are both, local small changes that can be made to 
# each of these functions as well as major changes in 
# legibility and usefullness of the code. This section
# will adress some of both.
#---------------------------------------------------------

import random
import numpy as np

data = np.load('temp_data.npy')

#---------------------------------------------------------
# when importing the data, the extra variable is unne-
# cessary and I moved the variables down to the main 
# section so they are better accessible when we actually
# need them.
#---------------------------------------------------------

def gen_zero(pop_size, num_of_columns):
    return np.random.choice(np.linspace(-1,80,num = 5000),size=(pop_size, num_of_columns),replace=False)

def error(coefficients):
    k = data.shape[0]
    return (np.sum([(data[j,1]-(coefficients[0] * data[j,0] + coefficients[1]))**2 for j in range(k)])) / k
    
def fitness_v(data, generation):
    return np.array([np.array([x,error(generation[x])]) for x in range(generation.shape[0])]) 

def survivor_sel(fit_vec, num_of_surv, generation):
    random_surv = np.random.choice(range(len(fit_vec)), num_of_surv/2, replace=False)
    best = np.argmin(fit_vec[random_surv,1])
    return generation[int(fit_vec[random_surv[best]][0])]

def crossover(num_new_pop, survivors):
    dupl_survivors = np.zeros((((num_new_pop.shape[0] - len(survivors))),survivors.shape[1]))
    for x in range(survivors.shape[1]):
        dupl_survivors[:, x] = np.repeat(survivors[:, x], 4, axis=0)
        dupl_survivors[:, x] = np.random.permutation(dupl_survivors[:, x])
    return dupl_survivors

#---------------------------------------------------------
# Matrices and arrays are essentially the same. While 
# matrices can do calculations that we can't do with 
# arrays, these are not needed in this context and thus
# converting the array here is unnecessary.
#---------------------------------------------------------

def mutation(population, mutat_rate):
    for n in population:
        if mutat_rate >= random.random():
            x_or_y = random.randint(0,1)
            n[x_or_y] = np.random.choice(np.linspace(-1,80, num = 1000), size = 1, replace = False)
    return population 

#---------------------------------------------------------
# A large change that I have made to all the functions is
# 1. naming them according to what they do and secondly
# removing the global variables that they call on. When 
# writing functions we always want to take global variables
# as imput and then assign them to local variable names in
# the function, which makes the functions more flexible. 
# In the original code changing just one variable name can
# render entire sections useless. These changes also 
# require some changes in the main section so that all 
# functions are called on the right variables.
#---------------------------------------------------------

initial_pop_size = 500
mutation_rate = 0.05
num_generations = 5
chromosome_length = 2
num_survivors = 100

current_pop = gen_zero(initial_pop_size, chromosome_length)
new_population = np.zeros((num_survivors * 5, current_pop.shape[1]))

for i in range(num_generations):
    f_vector = fitness_v(data, current_pop)
    survivors = np.zeros((num_survivors, chromosome_length))
    
    for n in range(len(survivors)):
        survivors[n] = survivor_sel(f_vector, num_survivors, current_pop)
        
    new_population[:len(survivors)] = survivors
    new_population[len(survivors):] = crossover(new_population, survivors)
    new_population = mutation(new_population, mutation_rate)
    
    current_pop = new_population
    new_population = np.zeros((num_survivors * 5, current_pop.shape[1]))

f_vector = fitness_v(data, current_pop)
best_solution = current_pop[np.argmin(fitness_vector[:,1])]
print "The best solution is", best_solution
print "with error equal to approximately", error(best_solution)

The best solution is [  0.5555111   70.02702703]
with error equal to approximately 23.7175484773
